# Rankings

In [ ]:
ENV="dev"

## Connection to PostgreSQL database

In [ ]:
from itables import init_notebook_mode, show

from postgres.PostgresToDataFrame import PostgresToDataframe
from src.rankings.TeamsRanking import TeamsRanking
from src.rankings.PlayersRanking import PlayersRanking

init_notebook_mode(all_interactive=True)

In [ ]:
db = PostgresToDataframe(ENV)

## Rankings

In [ ]:
ID_CHP: str = 'ligue_1'
SEASON: str = '2023-2024'
SIDE: str = 'both'
FIRST_WEEK: int = 1
LAST_WEEK: int = 100

N: int = 1000000
ROUND: int = 2

### Teams ranking

In [ ]:
teams_ranking = TeamsRanking(db)
t_ranking = teams_ranking.build_ranking(ID_CHP, SEASON, FIRST_WEEK, LAST_WEEK, SIDE, N, ROUND)

In [ ]:
show(t_ranking.sort_values('Points', ascending=False), paging=False)

In [ ]:
show(t_ranking[['Matches', 'Club', 'Wins', 'Draws', 'Loses', 'Points', 'xP', 'Diff Points']].set_index('Club').loc['FC Lorient'], paging=False)

### Players Rankings

In [ ]:
players_ranking = PlayersRanking(db)
p_ranking = players_ranking.build_ranking(ID_CHP, SEASON, FIRST_WEEK, LAST_WEEK, SIDE)

In [ ]:
p_ranking

## Distinct positions

In [ ]:
all_positions = db.df_from_query("select position from player_stats ps group by position;")["position"].dropna().tolist()
flattened_positions = []
for sublist in [position.split(',') for position in all_positions]:
    flattened_positions.extend(sublist)

In [ ]:
list(set(all_positions))

In [ ]:
unique_positions = list(set(flattened_positions))
general_positions = ['gk', 'def', 'mid', 'att']

{
    'gk': 'gk',
    'df': 'def',
    'lb': 'def',
    'cb': 'def',
    'wb': 'def', #wide back
    'rb': 'def',
    'dm': 'mid',
    'cm': 'mid',
    'lm': 'mid',
    'rm': 'mid',
    'mf': 'mid', #in-sub players
    'am': 'mid', #attacking midfielder
    'lw': 'att', #winger
    'rw': 'att',
    'fw': 'att'
}

# Closing the PostgreSQL session

In [ ]:
db.close()